In [2]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import optuna
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.lgbm.lgbm_cv_trainer as cv
import src.models.lgbm.lgbm_optuna_optimizer as op

In [3]:
# データの読み込み
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

tr_df1 = pd.read_csv("../artifacts/features/tr_df1.csv")
test_df1 = pd.read_csv("../artifacts/features/test_df1.csv")

In [6]:
# tr_df1のtuning
importlib.reload(cv)
importlib.reload(op)

params = {
    "max_depth": 5,
    "num_leaves": 523,
    "min_child_samples": 5056,
    "min_split_gain": 9.275294835907687e-05,
    "feature_fraction": 0.43593953046851464,
    "bagging_fraction": 0.8924361138693251,
    "bagging_freq": 10,
    "lambda_l1": 1.6934155410667961,
    "lambda_l2": 0.6637926838138378
}

objective = op.create_objective(
    tr_df1,
    early_stopping_rounds=500,
    n_jobs=25
)

random_sampler = optuna.samplers.RandomSampler(seed=42)

study = op.run_optuna_search(
    objective,
    n_trials=10,
    n_jobs=1,
    study_name="lgbm_v1",
    storage=url,
    sampler=random_sampler,
    initial_params=params
)

[I 2025-07-20 20:25:56,664] A new study created in RDB with name: lgbm_v1


  0%|          | 0/10 [00:00<?, ?it/s]

Training until validation scores don't improve for 500 rounds
[100]	train's rmse: 0.0671054	eval's rmse: 0.0664504
[200]	train's rmse: 0.0638185	eval's rmse: 0.063258
[300]	train's rmse: 0.062945	eval's rmse: 0.0624499
[400]	train's rmse: 0.0625537	eval's rmse: 0.0621067
[500]	train's rmse: 0.0622817	eval's rmse: 0.061874
[600]	train's rmse: 0.0620215	eval's rmse: 0.0616563
[700]	train's rmse: 0.0618602	eval's rmse: 0.0615309
[800]	train's rmse: 0.061722	eval's rmse: 0.0614297
[900]	train's rmse: 0.0615776	eval's rmse: 0.0613269
[1000]	train's rmse: 0.0614737	eval's rmse: 0.0612611
[1100]	train's rmse: 0.0613757	eval's rmse: 0.0611939
[1200]	train's rmse: 0.0612796	eval's rmse: 0.0611299
[1300]	train's rmse: 0.0611961	eval's rmse: 0.0610858
[1400]	train's rmse: 0.0611174	eval's rmse: 0.0610372
[1500]	train's rmse: 0.0610481	eval's rmse: 0.0609975
[1600]	train's rmse: 0.0609864	eval's rmse: 0.0609606
[1700]	train's rmse: 0.0609353	eval's rmse: 0.0609394
[1800]	train's rmse: 0.0608772	ev